In [ ]:
# from langchain.llms.openai import OpenAI # OpenAI 가 쓰는 방식이 ChatOpenAI 보다 비쌈
# UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI 라는 경고가 뜸 ㅋㅋ
# ChatOpenAI만 쓰세요~
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts import load_prompt
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.callbacks import StreamingStdOutCallbackHandler, get_openai_callback

In [2]:
chat = ChatOpenAI(
    temperature=0.1 # 창의성, 0~1 설정, 높으면 창의성 많음
)

In [19]:
template = PromptTemplate.from_template(
    "What is the distance betwwen {country_a} and {country_b}"
)

prompt = template.format(country_a="Mexico", country_b="Thailand")
prompt

'What is the distance betwwen Mexico and Thailand'

In [7]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}"),
    ("ai" ,"Ciao, mi chiamo {name}"),
    ("human", "What is the distance betwwen {country_a} and {country_b}. Also, what is your name?"),
])


In [33]:
prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a = "Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [4]:
from langchain.schema import BaseOutputParser

class CommnaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommnaOutputParser()

In [8]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "you ar a list generation machine. Everything you are asked will be answerd with a comma separated list of max {max_items} in lowercase. Do Not reply with anything else"),
        (
            "human",
            "{question}",
        ),
    ]
)

prompt = template.format_messages(max_items = 10, question = "What are the plenets")
result = chat.predict_messages(prompt)
p.parse(result.content)

['mercury',
 'venus',
 'earth',
 'mars',
 'jupiter',
 'saturn',
 'uranus',
 'neptune',
 'pluto']

In [6]:
chain = template | chat | CommnaOutputParser()

chain.invoke(
    {
        "max_items" : 5,
        "question" : "What are the plenets"
    }
)  

['mercury', 'venus', 'earth', 'mars', 'jupiter']

In [ ]:


chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a world-class international chef. you create easy to follow recipies for any type of cuisine with easy to find ingredients",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)
veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})

In [20]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:,
        Capital: Paris,
        Language: French,
        Food: Wine and Cheese,
        Currency: Euro,
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """,
        I know this:,
        Capital: Rome,
        Language: Italian,
        Food: Pizza and Pasta,
        Currency: Euro,
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """,
        I know this:,
        Capital: Athens,
        Language: Greek,
        Food: Souvlaki and Feta Cheese,
        Currency: Euro,
        """,
    },
]

In [24]:
example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({"country": "German"})

AI: 
        I know this:,
        Capital: Berlin,
        Language: German,
        Food: Bratwurst and Sauerkraut,
        Currency: Euro,

AIMessageChunk(content='AI: \n        I know this:,\n        Capital: Berlin,\n        Language: German,\n        Food: Bratwurst and Sauerkraut,\n        Currency: Euro,')

In [29]:
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:,
        Capital: Paris,
        Language: French,
        Food: Wine and Cheese,
        Currency: Euro,
        """,
    },
    {
        "country": "Italy",
        "answer": """,
        I know this:,
        Capital: Rome,
        Language: Italian,
        Food: Pizza and Pasta,
        Currency: Euro,
        """,
    },
    {
        "country": "Greece",
        "answer": """,
        I know this:,
        Capital: Athens,
        Language: Greek,
        Food: Souvlaki and Feta Cheese,
        Currency: Euro,
        """,
    },
]

In [33]:
example_prompt = ChatPromptTemplate.from_messages(
    [("human", "What do you know about {country}?"), ("ai", "{answer}")]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert"),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "German"})

,
        I know this:,
        Capital: Berlin,
        Language: German,
        Food: Bratwurst and Sauerkraut,
        Currency: Euro,
        

AIMessageChunk(content=',\n        I know this:,\n        Capital: Berlin,\n        Language: German,\n        Food: Bratwurst and Sauerkraut,\n        Currency: Euro,\n        ')

In [ ]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:,
        Capital: Paris,
        Language: French,
        Food: Wine and Cheese,
        Currency: Euro,
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """,
        I know this:,
        Capital: Rome,
        Language: Italian,
        Food: Pizza and Pasta,
        Currency: Euro,
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """,
        I know this:,
        Capital: Athens,
        Language: Greek,
        Food: Souvlaki and Feta Cheese,
        Currency: Euro,
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        super().__init__()
        self.examples = examples

    # 여기서 해당 함수를 구현해야하기 때문에 만들어 둔것, 기능 x
    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]


example_template = """
    Human: {question}
    AI: {answer}
"""
example_prompt = PromptTemplate.from_template(example_template)

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({"country": "German"})

AI: 
        Here is what I know:,
        Capital: Berlin,
        Language: German,
        Food: Bratwurst and Sauerkraut,
        Currency: Euro,

AIMessageChunk(content='AI: \n        Here is what I know:,\n        Capital: Berlin,\n        Language: German,\n        Food: Bratwurst and Sauerkraut,\n        Currency: Euro,')

In [48]:
prompt = load_prompt("./prompt.json")

prompt.format(country="Germany")

prompt = load_prompt("./prompt.yaml")

prompt.format(country="Germany")

'What is the capital of Germany'

In [49]:
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

In [58]:
prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

# 이건 테스트용
full_prompt.format(
    character="Pirate",
    example_question="What is your location?",
    example_answer="Arrrrg! That is a secret! Arg arg",
    question="What is your fav fod?",
)

chain = full_prompt | chat
chain.invoke({
    "character":"Pirate",
    "example_question":"What is your location?",
    "example_answer":"Arrrrg! That is a secret! Arg arg",
    "question":"What is your fav fod?",
})

Arrrrg! Me favorite food be a good ol' plate of fish 'n chips, matey! Nothing beats the taste of fresh seafood on the high seas! Arg arg!

AIMessageChunk(content="Arrrrg! Me favorite food be a good ol' plate of fish 'n chips, matey! Nothing beats the taste of fresh seafood on the high seas! Arg arg!")

In [65]:
set_llm_cache(InMemoryCache())

chat = ChatOpenAI(
    temperature=0.1
)

chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [66]:
chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [69]:
with get_openai_callback() as usage:
    chat.predict("What is the recipe for soju")
    print(usage)

Tokens Used: 222
	Prompt Tokens: 14
	Completion Tokens: 208
Successful Requests: 1
Total Cost (USD): $0.000437
